In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import matplotlib as plt
import seaborn as sns
from functools import reduce

In [2]:
seasons = range(2010,2024+1)
pbp = nfl.import_pbp_data(seasons)

2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.


# Team Play by Play Stat Calculations

- EPA Total
- EPA Per Play
- EPA per Pass
- EPA per Rush
- EPA per Sack
- EPA per Turnover
- Success Rate
- Success Rate Pass
- Success Rate Rush
- Third Down Conversion Rate
- Basic Team Totals: Pro Football Reference??
- Goal to Go Efficiency
- Play Action Efficiency
- Points per Drive
- Adjusted EPA?
- Drive Success Rate
- Next Gen Stats
- 

## EPA (Total, Per Play)

In [6]:
print(pbp.filter(like = 'sack').columns)

Index(['sack', 'lateral_sack_player_id', 'lateral_sack_player_name',
       'sack_player_id', 'sack_player_name', 'half_sack_1_player_id',
       'half_sack_1_player_name', 'half_sack_2_player_id',
       'half_sack_2_player_name'],
      dtype='object')


In [7]:
epa_pbp = pbp.query('season_type == "REG" &\
                    (`pass` == 1 | rush == 1) &\
                    epa.notnull() &\
                    qb_kneel != 1 &\
                    qb_spike != 1')

epa_team = epa_pbp\
    .groupby(['season', 'posteam'])\
    .agg({'epa': ['count', 'sum', 'mean']})

epa_team.columns = list(map('_'.join, epa_team.columns.values))

epa_team.reset_index(inplace = True)

epa_team = epa_team.rename(columns = {'epa_count': 'n',
                                      'epa_mean': 'epa_per_play',
                                      'epa_sum': 'total_epa'})

epa_team.query('season == 2024')\
        .sort_values('epa_per_play', ascending = False)\
        .head(32)

,season,posteam,n,total_epa,epa_per_play
450,2024,BAL,1091,215.262497,0.197308
451,2024,BUF,1065,199.021088,0.186874
458,2024,DET,1125,174.612961,0.155212
479,2024,WAS,1142,159.209152,0.139413
477,2024,TB,1128,148.944138,0.132043
473,2024,PHI,1129,124.859138,0.110593
454,2024,CIN,1121,119.292183,0.106416
459,2024,GB,1071,86.105789,0.080398
463,2024,KC,1124,77.364799,0.068830
448,2024,ARI,1087,73.809166,0.067902


## EPA per Dropback

In [9]:
epa_pbp_pass = pbp.query('season_type == "REG" &\
                         `pass` == 1 &\
                         passer_id.notnull() &\
                         epa.notnull() &\
                         qb_spike != 1')

epa_pass = epa_pbp_pass\
    .groupby(['season', 'posteam'])\
    .agg({'epa': ['count', 'sum', 'mean']})

epa_pass.columns = list(map('_'.join, epa_pass.columns.values))

epa_pass.reset_index(inplace = True)

epa_pass = epa_pass.rename(columns = {'epa_count': 'dropbacks',
                                      'epa_mean': 'epa_per_dropback',
                                      'epa_sum': 'total_pass_epa'})

epa_pass.query('season == 2024')\
        .sort_values('epa_per_dropback', ascending = False)\
        .head(32)

,season,posteam,dropbacks,total_pass_epa,epa_per_dropback
450,2024,BAL,577,186.857132,0.323843
451,2024,BUF,623,186.303421,0.299042
458,2024,DET,613,164.690781,0.268664
479,2024,WAS,690,145.015762,0.210168
477,2024,TB,689,143.716141,0.208587
454,2024,CIN,770,146.493591,0.190251
459,2024,GB,558,103.195999,0.184939
473,2024,PHI,565,92.227486,0.163234
476,2024,SF,642,90.296371,0.140649
463,2024,KC,723,100.722359,0.139312


## EPA per Rush

In [11]:
epa_pbp_rush = pbp.query('season_type == "REG" &\
                         rush == 1 &\
                         rusher_id.notnull() &\
                         epa.notnull()')

epa_rush = epa_pbp_rush\
    .groupby(['season', 'posteam'])\
    .agg({'epa': ['count', 'sum', 'mean']})

epa_rush.columns = list(map('_'.join, epa_rush.columns.values))

epa_rush.reset_index(inplace = True)

epa_rush = epa_rush.rename(columns = {'epa_count': 'n',
                                      'epa_mean': 'epa_per_rush',
                                      'epa_sum': 'total_rush_epa'})

epa_rush.query('season == 2024')\
        .sort_values('epa_per_rush', ascending = False)\
        .head(32)

,season,posteam,n,total_rush_epa,epa_per_rush
473,2024,PHI,564,32.631653,0.057858
450,2024,BAL,514,28.405363,0.055263
479,2024,WAS,452,14.193397,0.031401
451,2024,BUF,442,12.717658,0.028773
458,2024,DET,512,9.922181,0.019379
477,2024,TB,439,5.227996,0.011909
449,2024,ATL,490,-4.970702,-0.010144
448,2024,ARI,432,-8.471785,-0.019611
459,2024,GB,513,-17.090216,-0.033314
470,2024,NO,423,-15.046299,-0.035570


## EPA per Sack

In [13]:
epa_pbp_sack = pbp.query('season_type == "REG" &\
                         `pass` == 1 &\
                         sack == 1')

epa_sack = epa_pbp_sack.groupby(['season', 'posteam'])\
                       .agg({'epa': ['count', 'sum', 'mean']})

epa_sack.columns = list(map('_'.join, epa_sack.columns.values))

epa_sack.reset_index(inplace = True)

epa_sack = epa_sack.rename(columns = {'epa_count': 'n',
                                      'epa_sum': 'total_sack_epa',
                                      'epa_mean': 'epa_per_sack'})

epa_sack.sort_values('epa_per_sack', ascending = True).head(10)


,season,posteam,n,total_sack_epa,epa_per_sack
338,2020,LV,28,-71.862923,-2.566533
215,2016,NYG,22,-53.865044,-2.448411
304,2019,LA,22,-53.501495,-2.431886
365,2021,IND,32,-76.912170,-2.403505
311,2019,NYG,43,-103.292404,-2.402149
247,2017,NYG,35,-82.555061,-2.358716
336,2020,LA,25,-58.159813,-2.326392
433,2023,LAC,43,-99.995041,-2.325466
328,2020,DAL,44,-99.477493,-2.260852
375,2021,NYG,38,-85.671623,-2.254516


In [14]:
print(pbp.filter(like = 'success').columns)

Index(['series_success', 'success', 'xyac_success'], dtype='object')


## EPA per Turnover

In [16]:
epa_pbp_turnover = pbp.query('season_type == "REG" &\
                             (`pass` == 1 | rush == 1) &\
                             (interception == 1 | fumble_lost == 1)')

epa_turnover = epa_pbp_turnover.groupby(['season', 'posteam'])\
                               .agg({'epa': ['count', 'sum', 'mean']})

epa_turnover.columns = list(map('_'.join, epa_turnover.columns.values))

epa_turnover.reset_index(inplace = True)

epa_turnover = epa_turnover.rename(columns = {'epa_count': 'turnovers',
                                              'epa_sum': 'total_turnover_epa',
                                              'epa_mean': 'epa_per_turnover'})

epa_turnover.query('season == 2024').sort_values('epa_per_turnover', ascending = False).head(32)

,season,posteam,turnovers,total_turnover_epa,epa_per_turnover
462,2024,JAX,23,-75.084229,-3.264532
457,2024,DEN,19,-74.445290,-3.918173
448,2024,ARI,20,-81.007759,-4.050388
479,2024,WAS,14,-58.410240,-4.172160
460,2024,HOU,18,-78.845985,-4.380332
453,2024,CHI,14,-61.472324,-4.390880
465,2024,LAC,8,-35.141441,-4.392680
464,2024,LA,14,-62.416035,-4.458288
452,2024,CAR,22,-98.334412,-4.469746
477,2024,TB,23,-103.895180,-4.517182


# EPA by Pass Length

In [176]:
pass_plays = pbp.query('season_type == "REG" &\
                       `pass` == 1 &\
                       air_yards.notnull() &\
                       epa.notnull()')

def categorize_pass_length(air_yards):
    if air_yards <5:
        return 'short'
    elif 5 <= air_yards <=10:
        return 'medium'
    else:
        return 'long'
        
pass_plays['pass_length_category'] = pass_plays['air_yards'].apply(categorize_pass_length)

pass_length_epa_old = pass_plays.groupby(['season', 'posteam', 'pass_length_category'])\
                            .agg(total_plays = ('epa', 'count'),
                                 total_epa = ('epa', 'sum'),
                                 epa_per_play = ('epa', 'mean'))\
                            .reset_index()

pass_length_epa = pass_length_epa_old.pivot(index=['season','posteam'], columns='pass_length_category')

# Flatten MultiIndex columns
pass_length_epa.columns = [f"{stat}_{length}" for stat, length in pass_length_epa.columns]
pass_length_epa = pass_length_epa.reset_index()

pass_length_epa

C:\Users\conne\AppData\Local\Temp\ipykernel_12896\144120236.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pass_plays['pass_length_category'] = pass_plays['air_yards'].apply(categorize_pass_length)


,season,posteam,total_plays_long,total_plays_medium,total_plays_short,total_epa_long,total_epa_medium,total_epa_short,epa_per_play_long,epa_per_play_medium,epa_per_play_short
0,2010,ARI,207,148,204,14.313765,-39.900478,-68.436028,0.069149,-0.269598,-0.335471
1,2010,ATL,190,151,234,71.597893,53.944759,3.125275,0.376831,0.357250,0.013356
2,2010,BAL,167,129,194,92.207397,40.136261,-20.456562,0.552140,0.311134,-0.105446
3,2010,BUF,171,140,207,50.310192,4.947889,-45.225456,0.294212,0.035342,-0.218480
4,2010,CAR,128,119,235,13.339266,-0.068696,-91.723976,0.104213,-0.000577,-0.390315
...,...,...,...,...,...,...,...,...,...,...,...
475,2024,SEA,159,143,289,75.440788,14.869667,-8.813965,0.474470,0.103984,-0.030498
476,2024,SF,174,171,182,91.963135,47.083122,-12.363740,0.528524,0.275340,-0.067933
477,2024,TB,170,136,264,67.992332,54.575600,53.374718,0.399955,0.401291,0.202177
478,2024,TEN,163,131,233,71.090836,19.317389,-80.771057,0.436140,0.147461,-0.346657


## TEST 1: Merge all team epa tables into one

dataframes = [epa_team, epa_pass, epa_rush, epa_sack, epa_turnover] 

epa_merged = reduce(lambda left, right: pd.merge(left, 
                                                 right, 
                                                 on = ['season', 'posteam'], 
                                                 how = 'outer'), dataframes)

epa_merged.fillna(0, inplace = True)

epa_merged.query('season == 2024').sort_values('epa_per_play', ascending = False).head(32)

## Team Success Rate

In [20]:
pbp_sr = pbp.query('season_type == "REG" &\
                    (`pass` == 1 | rush == 1)')

sr_team = pbp_sr.groupby(['season', 'posteam'])\
                .agg({'success': ['mean', 'sum']})

sr_team.columns = list(map('_'.join, sr_team.columns))

sr_team.reset_index(inplace = True)

sr_team = sr_team.rename(columns = {'success_mean': 'success_rate_team',
                                    'success_sum': 'total_success_plays_team'})


sr_team.query('season == 2024').sort_values('success_rate_team', ascending = False).head(32)
                 

,season,posteam,success_rate_team,total_success_plays_team
458,2024,DET,0.496000,558.0
450,2024,BAL,0.494042,539.0
477,2024,TB,0.493794,557.0
449,2024,ATL,0.484472,546.0
479,2024,WAS,0.482487,551.0
454,2024,CIN,0.479929,538.0
451,2024,BUF,0.478873,510.0
463,2024,KC,0.475979,535.0
448,2024,ARI,0.473781,515.0
464,2024,LA,0.472998,508.0


## Pass Success Rate

In [36]:
pbp_sr_pass = pbp.query('season_type == "REG" &\
                        `pass` == 1 &\
                        qb_spike != 1')

sr_pass = pbp_sr_pass.groupby(['season', 'posteam'])\
                     .agg({'success': ['mean', 'sum']})

sr_pass.columns = list(map('_'.join, sr_pass.columns))

sr_pass.reset_index(inplace = True)

sr_pass = sr_pass.rename(columns = {'success_mean': 'success_rate_pass',
                                    'success_sum': 'total_success_plays_pass'})


sr_pass.query('season == 2024').sort_values('success_rate_pass', ascending = False).head(32)

,season,posteam,success_rate_pass,total_success_plays_pass
458,2024,DET,0.536705,329.0
454,2024,CIN,0.527273,406.0
477,2024,TB,0.525399,362.0
450,2024,BAL,0.523397,302.0
468,2024,MIN,0.507418,342.0
448,2024,ARI,0.503817,330.0
451,2024,BUF,0.500803,312.0
464,2024,LA,0.499208,315.0
476,2024,SF,0.498442,320.0
463,2024,KC,0.497925,360.0


## Rush Success Rate

In [38]:
pbp_sr_rush = pbp.query('season_type == "REG" &\
                         rush == 1')

sr_rush = pbp_sr_rush.groupby(['season', 'posteam'])\
                     .agg({'success': ['mean', 'sum']})

sr_rush.columns = list(map('_'.join, sr_rush.columns))

sr_rush.reset_index(inplace = True)

sr_rush = sr_rush.rename(columns = {'success_mean': 'success_rate_rush',
                                    'success_sum': 'total_success_plays_rush'})


sr_rush.query('season == 2024').sort_values('success_rate_rush', ascending = False).head(32)

,season,posteam,success_rate_rush,total_success_plays_rush
449,2024,ATL,0.475510,233.0
479,2024,WAS,0.462389,209.0
450,2024,BAL,0.461089,237.0
451,2024,BUF,0.447964,198.0
458,2024,DET,0.447266,229.0
477,2024,TB,0.444191,195.0
463,2024,KC,0.436409,175.0
464,2024,LA,0.435666,193.0
473,2024,PHI,0.429078,242.0
448,2024,ARI,0.428241,185.0


## First Down Success Rate

In [94]:
first_pbp_sr = pbp.query('season_type == "REG" &\
                         (`pass` == 1 | rush == 1) &\
                         down == 1')

first_down_sr = first_pbp_sr.groupby(['season', 'posteam'])\
                            .agg({'success': ['mean', 'sum']})

first_down_sr.columns = list(map('_'.join, first_down_sr.columns))

first_down_sr.reset_index(inplace = True)

first_down_sr = first_down_sr.rename(columns = {'success_mean': 'first_down_success_rate',
                                                'success_sum': 'first_down_success_total'})


first_down_sr.query('season == 2024').sort_values('first_down_success_rate', ascending = False).head(32)      

,season,posteam,first_down_success_rate,first_down_success_total
477,2024,TB,0.494094,251.0
458,2024,DET,0.489279,251.0
454,2024,CIN,0.479042,240.0
464,2024,LA,0.473795,226.0
450,2024,BAL,0.468504,238.0
449,2024,ATL,0.465070,233.0
463,2024,KC,0.450939,216.0
467,2024,MIA,0.450000,225.0
448,2024,ARI,0.449799,224.0
479,2024,WAS,0.444668,221.0


## Second Down Success Rate

In [97]:
second_pbp_sr = pbp.query('season_type == "REG" &\
                          (`pass` == 1 | rush == 1) &\
                          down == 2')

second_down_sr = second_pbp_sr.groupby(['season', 'posteam'])\
                              .agg({'success': ['mean', 'sum']})

second_down_sr.columns = list(map('_'.join, second_down_sr.columns))

second_down_sr.reset_index(inplace = True)

second_down_sr = second_down_sr.rename(columns = {'success_mean': 'second_down_success_rate',
                                                'success_sum': 'second_down_success_total'})


second_down_sr.query('season == 2024').sort_values('second_down_success_rate', ascending = False).head(32)    

,season,posteam,second_down_success_rate,second_down_success_total
451,2024,BUF,0.527066,185.0
450,2024,BAL,0.517711,190.0
448,2024,ARI,0.512465,185.0
449,2024,ATL,0.500000,184.0
479,2024,WAS,0.496124,192.0
468,2024,MIN,0.494565,182.0
458,2024,DET,0.493188,181.0
459,2024,GB,0.491848,181.0
464,2024,LA,0.481894,173.0
476,2024,SF,0.481050,165.0


## Third Down Success Rate

In [100]:
third_pbp_sr = pbp.query('season_type == "REG" &\
                         (`pass` == 1 | rush == 1) &\
                         down == 3')

third_down_sr = third_pbp_sr.groupby(['season', 'posteam'])\
                            .agg({'success': ['mean', 'sum']})

third_down_sr.columns = list(map('_'.join, third_down_sr.columns))

third_down_sr.reset_index(inplace = True)

third_down_sr = third_down_sr.rename(columns = {'success_mean': 'third_down_success_rate',
                                                'success_sum': 'third_down_success_total'})

third_down_sr.query('season == 2024').sort_values('third_down_success_rate', ascending = False).head(32)    

,season,posteam,third_down_success_rate,third_down_success_total
477,2024,TB,0.539130,124.0
463,2024,KC,0.531915,125.0
450,2024,BAL,0.522843,103.0
479,2024,WAS,0.495614,113.0
451,2024,BUF,0.490654,105.0
458,2024,DET,0.490476,103.0
454,2024,CIN,0.480000,108.0
449,2024,ATL,0.472973,105.0
472,2024,NYJ,0.466667,105.0
476,2024,SF,0.466346,97.0


## Fourth Down Success Rate

In [103]:
fourth_pbp_sr = pbp.query('season_type == "REG" &\
                         (`pass` == 1 | rush == 1) &\
                         down == 4')

fourth_down_sr = fourth_pbp_sr.groupby(['season', 'posteam'])\
                              .agg({'success': ['mean', 'sum']})

fourth_down_sr.columns = list(map('_'.join, fourth_down_sr.columns))

fourth_down_sr.reset_index(inplace = True)

fourth_down_sr = fourth_down_sr.rename(columns = {'success_mean': 'fourth_down_success_rate',
                                                  'success_sum': 'fourth_down_success_total'})

fourth_down_sr.query('season == 2024').sort_values('fourth_down_success_rate', ascending = False).head(32)    

,season,posteam,fourth_down_success_rate,fourth_down_success_total
479,2024,WAS,0.875000,21.0
457,2024,DEN,0.714286,15.0
451,2024,BUF,0.695652,16.0
449,2024,ATL,0.687500,22.0
473,2024,PHI,0.678571,19.0
458,2024,DET,0.666667,22.0
464,2024,LA,0.666667,16.0
460,2024,HOU,0.650000,13.0
465,2024,LAC,0.647059,11.0
463,2024,KC,0.619048,13.0


# Test 2: Combine

In [40]:
dataframes = [epa_team, epa_pass, epa_rush, epa_sack, epa_turnover, sr_team, sr_pass, sr_rush]

team_efficiency = reduce(lambda left, right: pd.merge(left, right, on = ['season', 'posteam'], how = 'outer'), dataframes)

team_efficiency.fillna(0, inplace = True)

team_efficiency.query('season == 2024').sort_values('epa_per_play', ascending = False).head(32)

,season,posteam,n_x,total_epa,epa_per_play,dropbacks,total_pass_epa,epa_per_dropback,n_y,total_rush_epa,...,epa_per_sack,turnovers,total_turnover_epa,epa_per_turnover,success_rate_team,total_success_plays_team,success_rate_pass,total_success_plays_pass,success_rate_rush,total_success_plays_rush
450,2024,BAL,1091,215.262497,0.197308,577,186.857132,0.323843,514,28.405363,...,-1.668336,11,-56.331287,-5.121026,0.494042,539.0,0.523397,302.0,0.461089,237.0
451,2024,BUF,1065,199.021088,0.186874,623,186.303421,0.299042,442,12.717658,...,-1.762314,8,-38.303204,-4.787900,0.478873,510.0,0.500803,312.0,0.447964,198.0
458,2024,DET,1125,174.612961,0.155212,613,164.690781,0.268664,512,9.922181,...,-1.776933,15,-68.804092,-4.586939,0.496000,558.0,0.536705,329.0,0.447266,229.0
479,2024,WAS,1142,159.209152,0.139413,690,145.015762,0.210168,452,14.193397,...,-1.377826,14,-58.410240,-4.172160,0.482487,551.0,0.495652,342.0,0.462389,209.0
477,2024,TB,1128,148.944138,0.132043,689,143.716141,0.208587,439,5.227996,...,-1.686836,23,-103.895180,-4.517182,0.493794,557.0,0.525399,362.0,0.444191,195.0
473,2024,PHI,1129,124.859138,0.110593,565,92.227486,0.163234,564,32.631653,...,-1.859513,12,-56.334473,-4.694540,0.452613,511.0,0.476106,269.0,0.429078,242.0
454,2024,CIN,1121,119.292183,0.106416,770,146.493591,0.190251,351,-27.201408,...,-2.006256,19,-92.836456,-4.886129,0.479929,538.0,0.527273,406.0,0.376068,132.0
459,2024,GB,1071,86.105789,0.080398,558,103.195999,0.184939,513,-17.090216,...,-1.557928,18,-90.207123,-5.011507,0.448179,480.0,0.476703,266.0,0.417154,214.0
463,2024,KC,1124,77.364799,0.068830,723,100.722359,0.139312,401,-23.357557,...,-1.484366,14,-63.699196,-4.549942,0.475979,535.0,0.497925,360.0,0.436409,175.0
448,2024,ARI,1087,73.809166,0.067902,655,82.280952,0.125620,432,-8.471785,...,-1.679053,20,-81.007759,-4.050388,0.473781,515.0,0.503817,330.0,0.428241,185.0


# Third Down Conversion Rate

In [67]:
third_down_pbp = pbp.query('season_type == "REG" &\
                            (`pass` == 1 | rush == 1) &\
                            down == 3 &\
                            timeout != 1 &\
                            qb_kneel != 1 &\
                            qb_spike != 1')

third_down_pbp.loc[:,'converted'] = third_down_pbp['yards_gained'] >= third_down_pbp['ydstogo']

conversion_rate = third_down_pbp\
    .groupby(['season', 'posteam'])\
    .agg({'converted': ['sum', 'count']})

conversion_rate.columns = list(map('_'.join, conversion_rate.columns.values))

conversion_rate.reset_index(inplace = True)

conversion_rate = conversion_rate.rename(columns = {'converted_sum': 'third_down_converted_total',
                                                    'converted_count': 'third_down_attempts'})

conversion_rate['third_down_conversion_rate'] = conversion_rate['third_down_converted_total'] / conversion_rate['third_down_attempts']

conversion_rate.query('season == 2024').sort_values('third_down_conversion_rate', ascending = False).head(32)

C:\Users\conne\AppData\Local\Temp\ipykernel_12896\3278985898.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  third_down_pbp.loc[:,'converted'] = third_down_pbp['yards_gained'] >= third_down_pbp['ydstogo']


,season,posteam,third_down_converted_total,third_down_attempts,third_down_conversion_rate
477,2024,TB,111,230,0.482609
463,2024,KC,110,233,0.472103
458,2024,DET,97,208,0.466346
450,2024,BAL,91,196,0.464286
454,2024,CIN,102,225,0.453333
479,2024,WAS,100,227,0.440529
473,2024,PHI,98,237,0.413502
451,2024,BUF,88,214,0.411215
448,2024,ARI,83,202,0.410891
476,2024,SF,83,206,0.402913


## Red Zone Conversion Rate

In [180]:
redzone_pbp = pbp.query('season_type == "REG" &\
                        (`pass` == 1 | rush == 1) &\
                        yardline_100 <= 20 &\
                        down.notnull()')

redzone_tds = redzone_pbp.query('touchdown == 1')

redzone_stats = redzone_pbp\
    .groupby(['season', 'posteam'])\
    .agg(redzone_trips = ('drive', 'nunique'))\
    .reset_index()

td_stats = redzone_tds\
    .groupby(['season', 'posteam'])\
    .agg(redzone_tds = ('touchdown', 'count'))\
    .reset_index()

redzone_tds.reset_index()

redzone_summary = redzone_stats.merge(td_stats, on=['season', 'posteam'], how='left')
redzone_summary['redzone_tds'] = redzone_summary['redzone_tds'].fillna(0)
redzone_summary['redzone_td_rate'] = redzone_summary['redzone_tds'] / redzone_summary['redzone_trips']

# Sort by highest red zone TD rate
redzone_summary = redzone_summary.sort_values('redzone_td_rate', ascending=False)

# View result
redzone_summary.head(10)

,season,posteam,redzone_trips,redzone_tds,redzone_td_rate
271,2018,KC,20,51,2.550000
392,2022,DAL,17,42,2.470588
200,2016,DAL,16,39,2.437500
458,2024,DET,21,51,2.428571
399,2022,KC,22,51,2.318182
105,2013,DEN,24,55,2.291667
342,2020,NO,23,52,2.260870
331,2020,GB,22,48,2.181818
451,2024,BUF,22,48,2.181818
450,2024,BAL,23,50,2.173913


In [182]:
# Step 1: Get all red zone plays
redzone_pbp = pbp.query('season_type == "REG" & \
                         (`pass` == 1 | rush == 1) & \
                         yardline_100 <= 20 & \
                         down.notnull()')

# Step 2: Get 1 row per red zone trip (first red zone play per drive)
redzone_drives = redzone_pbp.sort_values(['game_id', 'drive', 'play_id'])\
                             .drop_duplicates(subset=['season', 'posteam', 'game_id', 'drive'])

# Step 3: Add redzone touchdown column (1 if that drive had a TD at any point)
# First, get drives with any TD
td_drives = redzone_pbp[redzone_pbp['touchdown'] == 1]\
    .drop_duplicates(subset=['game_id', 'drive'])[['game_id', 'drive']]
td_drives['redzone_td'] = 1

# Merge to mark which redzone drives had a TD
redzone_trips = redzone_drives.merge(td_drives, on=['game_id', 'drive'], how='left')
redzone_trips['redzone_td'] = redzone_trips['redzone_td'].fillna(0)

# Step 4: Group and calculate red zone conversion rate
redzone_summary = redzone_trips.groupby(['season', 'posteam']).agg(
    redzone_trips=('drive', 'count'),
    redzone_tds=('redzone_td', 'sum')
).reset_index()

redzone_summary['redzone_td_rate'] = redzone_summary['redzone_tds'] / redzone_summary['redzone_trips']

# View top 10
redzone_summary.query('season == 2024').sort_values('redzone_td_rate', ascending=False).head(32)


,season,posteam,redzone_trips,redzone_tds,redzone_td_rate
450,2024,BAL,67,50.0,0.746269
451,2024,BUF,67,48.0,0.716418
458,2024,DET,72,51.0,0.708333
477,2024,TB,66,45.0,0.681818
454,2024,CIN,61,39.0,0.639344
453,2024,CHI,36,23.0,0.638889
479,2024,WAS,71,45.0,0.633803
457,2024,DEN,58,36.0,0.620690
459,2024,GB,68,42.0,0.617647
475,2024,SEA,45,27.0,0.600000


# Explosive Play Percent

In [208]:
ep_pbp = pbp.query('season_type == "REG" &\
                    (rush == 1 | `pass` == 1) &\
                    yards_gained.notnull()')

ep_pbp['explosive'] = ((ep_pbp['pass'] == 1) & (ep_pbp['yards_gained'] >= 20)) | \
                      ((ep_pbp['rush'] == 1) & (ep_pbp['yards_gained'] >= 10))

ep_pbp['explosive_pass'] = (ep_pbp['pass'] == 1) & ep_pbp['explosive']
ep_pbp['explosive_rush'] = (ep_pbp['rush'] == 1) & ep_pbp['explosive']

explosive_plays = ep_pbp.groupby(['season', 'posteam'])\
                        .agg(total_plays          = ('play_id', 'count'),
                             total_pass_plays     = ('pass', 'sum'),
                             total_rush_plays     = ('rush', 'sum'),
                             explosive_plays      = ('explosive', 'sum'),
                             explosive_pass_plays = ('explosive_pass', 'sum'),
                             explosive_rush_plays = ('explosive_rush', 'sum'))\
                        .reset_index()

explosive_plays['explosive_play_rate'] = explosive_plays['explosive_plays'] / explosive_plays['total_plays']
explosive_plays['explosive_pass_rate'] = explosive_plays['explosive_pass_plays'] / explosive_plays['total_pass_plays']
explosive_plays['explosive_rush_rate'] = explosive_plays['explosive_rush_plays'] / explosive_plays['total_rush_plays']

explosive_plays.query('season == 2024')\
               .sort_values('explosive_play_rate', ascending = False)\
               .head(32)

C:\Users\conne\AppData\Local\Temp\ipykernel_12896\387514866.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ep_pbp['explosive'] = ((ep_pbp['pass'] == 1) & (ep_pbp['yards_gained'] >= 20)) | \
C:\Users\conne\AppData\Local\Temp\ipykernel_12896\387514866.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ep_pbp['explosive_pass'] = (ep_pbp['pass'] == 1) & ep_pbp['explosive']
C:\Users\conne\AppData\Local\Temp\ipykernel_12896\387514866.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,season,posteam,total_plays,total_pass_plays,total_rush_plays,explosive_plays,explosive_pass_plays,explosive_rush_plays,explosive_play_rate,explosive_pass_rate,explosive_rush_rate
450,2024,BAL,1088,574.0,514.0,131,60,71,0.120404,0.104530,0.138132
459,2024,GB,1068,555.0,513.0,125,61,64,0.117041,0.109910,0.124756
458,2024,DET,1125,613.0,512.0,129,62,67,0.114667,0.101142,0.130859
473,2024,PHI,1127,563.0,564.0,120,55,65,0.106477,0.097691,0.115248
476,2024,SF,1064,641.0,423.0,111,66,45,0.104323,0.102964,0.106383
461,2024,IND,1074,604.0,470.0,110,55,55,0.102421,0.091060,0.117021
477,2024,TB,1127,688.0,439.0,115,60,55,0.102041,0.087209,0.125285
448,2024,ARI,1086,654.0,432.0,108,47,61,0.099448,0.071865,0.141204
474,2024,PIT,1110,619.0,491.0,109,55,54,0.098198,0.088853,0.109980
449,2024,ATL,1127,637.0,490.0,109,54,55,0.096717,0.084772,0.112245


In [189]:
print(pbp.filter(like = 'yards').columns)

Index(['yards_gained', 'air_yards', 'yards_after_catch', 'passing_yards',
       'receiving_yards', 'rushing_yards', 'lateral_receiving_yards',
       'lateral_rushing_yards', 'fumble_recovery_1_yards',
       'fumble_recovery_2_yards', 'return_yards', 'penalty_yards',
       'drive_yards_penalized', 'ngs_air_yards'],
      dtype='object')


# Final Combination

In [210]:
dataframes = [epa_team, 
              epa_pass, 
              epa_rush, 
              epa_sack, 
              epa_turnover, 
              pass_length_epa, 
              sr_team, 
              sr_pass, 
              sr_rush,
              first_down_sr,
              second_down_sr,
              third_down_sr,
              fourth_down_sr,
              conversion_rate,
              redzone_summary,
              explosive_plays]

team_efficiency = reduce(lambda left, right: pd.merge(left, right, on = ['season', 'posteam'], how = 'outer'), dataframes)

team_efficiency.fillna(0, inplace = True)

team_efficiency.query('season == 2024').sort_values('epa_per_play', ascending = False).head(32)

,season,posteam,n_x,total_epa,epa_per_play,dropbacks,total_pass_epa,epa_per_dropback,n_y,total_rush_epa,...,redzone_td_rate,total_plays,total_pass_plays,total_rush_plays,explosive_plays,explosive_pass_plays,explosive_rush_plays,explosive_play_rate,explosive_pass_rate,explosive_rush_rate
450,2024,BAL,1091,215.262497,0.197308,577,186.857132,0.323843,514,28.405363,...,0.746269,1088,574.0,514.0,131,60,71,0.120404,0.104530,0.138132
451,2024,BUF,1065,199.021088,0.186874,623,186.303421,0.299042,442,12.717658,...,0.716418,1062,620.0,442.0,101,60,41,0.095104,0.096774,0.092760
458,2024,DET,1125,174.612961,0.155212,613,164.690781,0.268664,512,9.922181,...,0.708333,1125,613.0,512.0,129,62,67,0.114667,0.101142,0.130859
479,2024,WAS,1142,159.209152,0.139413,690,145.015762,0.210168,452,14.193397,...,0.633803,1140,689.0,451.0,97,50,47,0.085088,0.072569,0.104213
477,2024,TB,1128,148.944138,0.132043,689,143.716141,0.208587,439,5.227996,...,0.681818,1127,688.0,439.0,115,60,55,0.102041,0.087209,0.125285
473,2024,PHI,1129,124.859138,0.110593,565,92.227486,0.163234,564,32.631653,...,0.550725,1127,563.0,564.0,120,55,65,0.106477,0.097691,0.115248
454,2024,CIN,1121,119.292183,0.106416,770,146.493591,0.190251,351,-27.201408,...,0.639344,1119,768.0,351.0,82,50,32,0.073280,0.065104,0.091168
459,2024,GB,1071,86.105789,0.080398,558,103.195999,0.184939,513,-17.090216,...,0.617647,1068,555.0,513.0,125,61,64,0.117041,0.109910,0.124756
463,2024,KC,1124,77.364799,0.068830,723,100.722359,0.139312,401,-23.357557,...,0.546875,1118,719.0,399.0,65,44,21,0.058140,0.061196,0.052632
448,2024,ARI,1087,73.809166,0.067902,655,82.280952,0.125620,432,-8.471785,...,0.592593,1086,654.0,432.0,108,47,61,0.099448,0.071865,0.141204


In [212]:
team_efficiency.to_csv('team_efficiency_stats.csv', index = False)